# 模型存檔與載入(Save and Loading)

## MNIST 手寫阿拉伯數字辨識

In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

# 載入 MNIST 手寫阿拉伯數字資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

# 建立模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 模型訓練
history = model.fit(x_train_norm, y_train, epochs=5, validation_split=0.2)

# 評分(Score Model)
score=model.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4575 - accuracy: 0.8651 - val_loss: 0.1314 - val_accuracy: 0.9627
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1312 - accuracy: 0.9610 - val_loss: 0.0963 - val_accuracy: 0.9711
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0937 - accuracy: 0.9731 - val_loss: 0.0892 - val_accuracy: 0.9741
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0681 - accuracy: 0.9784 - val_loss: 0.0812 - val_accuracy: 0.9768
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0542 - accuracy: 0.9824 - val_loss: 0.0791 - val_accuracy: 0.9768
loss: 0.0732
accuracy: 0.9769


## 模型存檔(Save)

In [2]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model\assets


## 模型載入(Loading)

In [10]:
# 模型載入
model2 = tf.keras.models.load_model('my_model')

# 評分(Score Model)
score=model2.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model2.metrics_names[i]}: {score[i]:.4f}')

loss: 0.0834
accuracy: 0.9750


In [11]:
# 模型比較
import numpy as np

# 比較，若結果不同，會出現錯誤
np.testing.assert_allclose(
    model.predict(x_test_norm), model2.predict(x_test_norm)
)

## Keras h5 模型存檔(Save)

In [2]:
model.save('my_h5_model.h5')

In [3]:
# 模型載入
model3 = tf.keras.models.load_model('my_h5_model.h5')

## 取得模型結構

In [4]:
# 取得模型結構
config = model.get_config()

# 載入模型結構
# Sequential model
new_model = tf.keras.Sequential.from_config(config)

# function API
# new_model = tf.keras.Model.from_config(config)

## json 格式

In [5]:
# 取得模型結構
json_config = model.to_json()

# 載入模型結構
new_model = tf.keras.models.model_from_json(json_config)

## 取得模型權重

In [6]:
# 取得模型權重
weights = model.get_weights()
weights

[array([[-0.05122572, -0.04820989,  0.01318664, ...,  0.03143455,
          0.00297445, -0.07300355],
        [ 0.0101722 ,  0.03137897, -0.03661704, ..., -0.0711699 ,
         -0.01441861, -0.0257089 ],
        [ 0.02733881, -0.0616094 , -0.0261831 , ..., -0.0515497 ,
         -0.07041344,  0.02369354],
        ...,
        [ 0.06716409, -0.02144293,  0.01964677, ...,  0.0598575 ,
         -0.06414444, -0.03448999],
        [ 0.05431178, -0.07536372, -0.03489753, ...,  0.05289243,
          0.04831088,  0.06894174],
        [ 0.0050236 , -0.03827769, -0.07115825, ..., -0.07327431,
          0.00056756,  0.05619989]], dtype=float32),
 array([-0.03355365,  0.035563  , -0.02280897, -0.02861411,  0.01783536,
         0.01770549, -0.04224253,  0.00521078,  0.02733752,  0.0242949 ,
         0.05571995,  0.06856805, -0.0332422 ,  0.0560267 , -0.0073974 ,
         0.03009453, -0.06499308, -0.02039569,  0.01326195,  0.05001385,
        -0.09214851,  0.04905553,  0.03864126,  0.08663406,  0.021

In [23]:
# 設定模型權重
new_model.set_weights(weights)

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
new_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# predict
score=new_model.evaluate(x_test_norm, y_test, verbose=0)
score

[0.08337192982435226, 0.9750000238418579]

## 取得模型結構時，Custom Layer 需註冊

In [30]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config


def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# Make a model with the CustomLayer and custom_activation
inputs = tf.keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = tf.keras.layers.Activation(custom_activation)(x)
model = tf.keras.Model(inputs, outputs)

# Retrieve the config
config = model.get_config()

# Custom Layer 需註冊
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
with tf.keras.utils.custom_object_scope(custom_objects):
    new_model = tf.keras.Model.from_config(config)

In [8]:
# 模型權重存檔，有 Custom Layer 會出現錯誤
model.save_weights('my_h5_model.weight')

In [9]:
# 載入模型權重檔
model.load_weights('my_h5_model.weight')